In [ ]:
pip install procgen

# Permet d'afficher un GIF

In [ ]:
import torch
from procgen import ProcgenEnv
from procgen_wrappers import VecExtractDictObs, TransposeFrame, ScaledFloatFrame

env = ProcgenEnv(
        num_envs=1,
        env_name="fruitbot",
        start_level=0,
        num_levels=100,
        distribution_mode='easy',
    )

env = VecExtractDictObs(env, "rgb")
env = TransposeFrame(env)
env = ScaledFloatFrame(env)


In [ ]:
from agent import Agent

agent = Agent().cuda()
agent.load_state_dict(torch.load('agent_weights.pth'))
agent.eval()


In [ ]:

import imageio
from IPython.display import Image
from tqdm.notebook import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np

def obs_to_image(obs):
  return (obs[0].transpose(1,2,0) * 255).astype(np.uint8)

def display_trajectory(frames, name, fps=25):
  imageio.mimwrite('./' + name,
                [obs_to_image(frame) for i, frame in enumerate(frames)],
                fps=fps)
  #return(Image(open('tmp.gif','rb').read(), width=500, height=500))

frames = []
obs = env.reset()

while True:
    frames.append(obs)
    obs = torch.FloatTensor(obs).to('cuda')
    action = agent(obs).argmax(1).cpu().numpy()
    obs, _, done ,_ = env.step(action)
    img = env.render()
    if done[0]:
        break

env.close()

display_trajectory(frames, "run.gif")

**Choisir et mettre en oeuvre 3 méthodes !**

# Méthode 1 : Vanilla Gradient

In [ ]:
def grad_to_image(grad):
  return (grad * 255).astype(np.uint8)

def obs_to_image(obs):
  return (obs[0].transpose(1,2,0) * 255).astype(np.uint8)

def display_trajectory_grad(frames, name, fps=25):
  imageio.mimwrite('./' + name,
                [grad_to_image(frame)[0] for i, frame in enumerate(frames)],
                fps=fps)
  #return(Image(open('grad.gif','rb').read(), width=500, height=500))

In [ ]:

obs = env.reset()
frames_1 = []
frames_vanilla_grad = []
while True:
    frames_1.append(obs)
    img = torch.from_numpy(obs[0].astype(np.float32))
    obs = torch.FloatTensor(obs).to('cuda')

    img = img.unsqueeze(0).cuda()
    img.requires_grad_()

    output = agent(img)
    output_idx = output.argmax()
    output_max = output[0, output_idx]

    output_max.backward()

    saliency, _ = torch.max(img.grad.data.abs(), dim=1)
    #saliency = saliency.squeeze(0)


    frames_vanilla_grad.append(saliency.cpu().numpy())

    action = agent(obs).argmax(1).cpu().numpy()
    obs, _, done ,_ = env.step(action)

    if done[0]:
        break

env.close()

display_trajectory(frames_1, "framesv1.gif")
display_trajectory_grad(frames_vanilla_grad, "vanilla_grad.gif")


In [ ]:
# On affiche le gif

from matplotlib.animation import FuncAnimation
%matplotlib inline

# Spécifiez les chemins des fichiers GIF
chemin_fichier_gif1 = 'framesv1.gif'
chemin_fichier_gif2 = 'vanilla_grad.gif'

# Lire les GIF
images_gif1 = imageio.mimread(chemin_fichier_gif1)
images_gif2 = imageio.mimread(chemin_fichier_gif2)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Initialiser les images des sous-plots
img1 = axs[0].imshow(images_gif1[0])
img2 = axs[1].imshow(images_gif2[0])

# Fonction d'animation
def update(frame):
    img1.set_array(images_gif1[frame])
    img2.set_array(images_gif2[frame])
    return img1, img2

# Créer l'animation
num_frames = min(len(images_gif1), len(images_gif2))
animation = FuncAnimation(fig, update, frames=num_frames, interval=100, blit=True)

# Afficher l'animation
from IPython.display import HTML
HTML(animation.to_jshtml())

# Méthode 2 : Smooth Gradient

In [ ]:
def get_vanilla_grad(img, model):
  img.retain_grad()
  output = model(img)
  output_idx = output.argmax()
  output_max = output[0, output_idx]
  output_max.backward()

  return img.grad


def grad_to_image(grad):
  return (grad * 255).astype(np.uint8)



In [ ]:
"""
obs = env.reset()


img = torch.from_numpy(obs[0].astype(np.float32))
img.requires_grad_()
obs = torch.FloatTensor(obs).to('cuda')

img = img.to('cuda').unsqueeze(0)

grad = get_vanilla_grad(img, agent)

saliency, _ = torch.max(abs(grad), dim=1)
saliency = saliency.squeeze(0)

plt.figure()
plt.imshow(saliency.cpu(), cmap='hot')
plt.axis('off')

"""


In [ ]:
# On calcule

obs = env.reset()
frames_2 = []
frames_smooth_grad = []
while True:
    frames_2.append(obs)
    img = torch.from_numpy(obs[0].astype(np.float32))
    img.requires_grad_()
    obs = torch.FloatTensor(obs).to('cuda')

    img = img.to('cuda').unsqueeze(0)

    grad = get_vanilla_grad(img, agent)

    saliency, _ = torch.max(abs(grad), dim=1)
    #saliency = saliency.squeeze(0)


    frames_smooth_grad.append(saliency.cpu().numpy())

    action = agent(obs).argmax(1).cpu().numpy()
    obs, _, done ,_ = env.step(action)

    if done[0]:
        break

env.close()



In [ ]:
#On sauvegarde sous forme de gif

display_trajectory(frames_2, "framesv2.gif")
display_trajectory_grad(frames_smooth_grad, 'grad_smooth.gif')

In [ ]:
# On affiche le gif

from matplotlib.animation import FuncAnimation
%matplotlib inline

# Spécifiez les chemins des fichiers GIF
chemin_fichier_gif1 = 'framesv2.gif'
chemin_fichier_gif2 = 'grad_smooth.gif'

# Lire les GIF
images_gif1 = imageio.mimread(chemin_fichier_gif1)
images_gif2 = imageio.mimread(chemin_fichier_gif2)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Initialiser les images des sous-plots
img1 = axs[0].imshow(images_gif1[0])
img2 = axs[1].imshow(images_gif2[0])

# Fonction d'animation
def update(frame):
    img1.set_array(images_gif1[frame])
    img2.set_array(images_gif2[frame])
    return img1, img2

# Créer l'animation
num_frames = min(len(images_gif1), len(images_gif2))
animation = FuncAnimation(fig, update, frames=num_frames, interval=100, blit=True)

# Afficher l'animation
from IPython.display import HTML
HTML(animation.to_jshtml())

# Méthode 3

In [ ]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import torch
import os
from torch.utils.data import Dataset
import cv2


means, stds = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
train_transform = transforms.Compose([
 transforms.Resize((224, 224)),
 transforms.ToTensor(),
 transforms.Normalize(means, stds),
 ])

test_transform = transforms.Compose([
 transforms.Resize((224, 224)),
 transforms.ToTensor(),
 transforms.Normalize(means, stds),
 ])

inv_normalize = transforms.Normalize(
 mean= [-m/s for m, s in zip(means, stds)],
 std= [1/s for s in stds]
)



In [ ]:

class HookFeatures():
  def __init__(self, module):
    self.feature_hook = module.register_forward_hook(self.feature_hook_fn)
  def feature_hook_fn(self, module, input, output):
    self.features = output.clone().detach()
    self.gradient_hook = output.register_hook(self.gradient_hook_fn)
  def gradient_hook_fn(self, grad):
    self.gradients = grad
  def close(self):
    self.feature_hook.remove()
    self.gradient_hook.remove()

hook = HookFeatures(agent.features)


In [ ]:
done = False
obs = env.reset()

frames_3 = []
frames_hook = []

while not done :
  frames_3.append(obs) # Verifier que c'est au bon endroit
  obs = torch.tensor(obs,requires_grad=True).to('cuda').float()
  output = agent(obs)
  output_idx = output.argmax()
  output_max = output[0, output_idx]
  output_max.backward()
  action = output.argmax(dim=1)


  gradients = hook.gradients
  activations = hook.features
  pooled_gradients = torch.mean(gradients, dim=[0, 2, 3]) # we take the average gradient of every chanels

  for i in range(activations.shape[1]):
      activations[:, i, :, :] *= pooled_gradients[i] # we multiply every chanels of the feature map with their corresponding averaged gradients

  obs, _, done ,_ = env.step(action.cpu().numpy())


  heatmap = torch.mean(activations, dim=1).squeeze()
  heatmap = np.maximum(heatmap.detach().cpu(), 0)
  heatmap /= torch.max(heatmap)
  heatmap = cv2.resize(np.float32(heatmap), (obs.shape[2], obs.shape[3]))
  heatmap = np.uint8(255 * heatmap)
  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_RAINBOW) / 255
  superposed_img = (heatmap) * 0.4 + obs[0].transpose(1, 2, 0)
  result = np.clip(superposed_img,0,1)
  frames_hook.append(result)


In [ ]:
#On affiche et sauvegarde sous forme de gif
from PIL import Image as im

def nparray_to_image(obs):
  return (obs * 255).astype(np.uint8)

def display_trajectory_hook(frames, name, fps=25):

  imageio.mimwrite('./' + name,
                [nparray_to_image(frame) for i, frame in enumerate(frames)],
                fps=fps)


display_trajectory(frames_3, "framesv3.gif")
display_trajectory_hook(frames_hook, 'hook.gif')

In [ ]:
chemin_fichier_gif1 = 'hook.gif'
images_gif1 = imageio.mimread(chemin_fichier_gif1)

In [ ]:
# On affiche le gif

from matplotlib.animation import FuncAnimation
%matplotlib inline

# Spécifiez les chemins des fichiers GIF
chemin_fichier_gif1 = 'framesv3.gif'
chemin_fichier_gif2 = 'hook.gif'

# Lire les GIF
images_gif1 = imageio.mimread(chemin_fichier_gif1)
images_gif2 = imageio.mimread(chemin_fichier_gif2)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Initialiser les images des sous-plots
img1 = axs[0].imshow(images_gif1[0])
img2 = axs[1].imshow(images_gif2[0])

# Fonction d'animation
def update(frame):
    img1.set_array(images_gif1[frame])
    img2.set_array(images_gif2[frame])
    return img1, img2

# Créer l'animation
num_frames = min(len(images_gif1), len(images_gif2))
animation = FuncAnimation(fig, update, frames=num_frames, interval=100, blit=True)

# Afficher l'animation
from IPython.display import HTML
HTML(animation.to_jshtml())